# 📊 PyPortfolio - Otimização de Portfólio v2.5**Markowitz, Fronteira Eficiente, Monte Carlo e Sharpe vs CDI**---### Changelog v2.5- ✅ **CORREÇÃO CRÍTICA:** Mapeamento de pesos ANTES do filtro MIN_OBS- ✅ **Status de mapeamento:** OK / REVISAR / NAO_MAPEADO / MAPEADO_EXCLUIDO_MIN_OBS- ✅ **Tabelas estruturadas:** Métricas das carteiras + Pesos por ativo- ✅ **Checagem de sucesso do solver** com fallback para equal-weight- ✅ **MIN_OVERLAP efetivo** na covariância (zera pares com overlap baixo)- ✅ Relatório de mapeamento detalhado em DataFrame### Bug corrigido:- Antes: se aba era excluída por MIN_OBS, o fuzzy match remapeava para OUTRO ativo (erro grave)- Agora: peso fica como "MAPEADO_EXCLUIDO_MIN_OBS" e é reportado (não remapeia)

---## 🔧 BLOCO 0 - CONFIGURAÇÕES GLOBAIS

In [ ]:
# =============================================================================# CONFIGURAÇÕES GLOBAIS - EDITE AQUI# =============================================================================# --- Caminhos de Arquivos ---EXCEL_DIR = "1 - Dados/1 - Rentabilidade atual"EXCEL_PATTERN = "Rendimentos_Mensais_Ativos*.xlsx"# --- Período de Análise ---DATA_INICIO = "2020-01-01"DATA_FIM = "2026-01-01"# --- Parâmetros de Qualidade de Dados ---MIN_OBS = 12              # Mínimo de meses por ativoMIN_OVERLAP = 12          # Mínimo de meses em comum para covariânciaVOL_FLOOR_MENSAL = 0.005  # 0.5% ao mês (~1.7% a.a.)# --- Ativos a excluir do universo ---ATIVOS_EXCLUIR = ["USDC"]# --- Thresholds de mapeamento ---THRESHOLD_OK = 0.90       # score >= 0.90: aceita automaticamenteTHRESHOLD_REVISAR = 0.75  # 0.75 <= score < 0.90: aceita mas marca REVISAR# score < 0.75: NAO_MAPEADO# --- Taxa Livre de Risco ---RF_MODO = "media_periodo"RF_MANUAL = 0.1150RF_FALLBACK = 0.10# --- Parâmetros de Otimização ---PESO_MAX_ATIVO = 1.0PESO_MIN_ATIVO = 0.0# --- Monte Carlo ---NUM_PORTFOLIOS = 80000RANDOM_SEED = 42MC_ALPHA_CONC = 0.2MC_ALPHA_DIV = 1.0MC_FRAC_CANTOS = 0.05MC_FRAC_SPARSE = 0.15# --- Fronteira Eficiente ---N_PONTOS_FRONTEIRA = 60# --- Retorno ---USAR_MEDIA_GEOMETRICA = True# --- Visualização ---TOP_N_PESOS = 20print("✅ Configurações carregadas!")print(f"   MIN_OBS: {MIN_OBS} meses | MIN_OVERLAP: {MIN_OVERLAP} meses")print(f"   Thresholds: OK >= {THRESHOLD_OK}, REVISAR >= {THRESHOLD_REVISAR}")

---## 📦 BLOCO 1 - BIBLIOTECAS E FUNÇÕES UTILITÁRIAS

In [ ]:
# =============================================================================# IMPORTS# =============================================================================import pandas as pdimport numpy as npfrom scipy.optimize import minimizefrom scipy.linalg import eighimport plotly.graph_objects as goimport warningsfrom pathlib import Pathfrom difflib import SequenceMatcherimport refrom typing import Optional, Dict, Tuple, Listfrom IPython.display import displayimport globtry:    import yfinance as yf    YFINANCE_DISPONIVEL = Trueexcept ImportError:    YFINANCE_DISPONIVEL = Falsewarnings.filterwarnings('ignore', category=FutureWarning)pd.set_option('display.max_columns', 20)pd.set_option('display.max_rows', 50)print("✅ Bibliotecas importadas!")

In [ ]:
# =============================================================================# FUNÇÕES - BUSCA E SELEÇÃO DE ARQUIVO EXCEL# =============================================================================def encontrar_arquivos_excel(diretorio: str, pattern: str = "*.xlsx") -> List[Path]:    """Encontra arquivos Excel no diretório que correspondem ao padrão."""    dir_path = Path(diretorio)    if not dir_path.exists():        return []    arquivos = list(dir_path.glob(pattern))    arquivos.sort(key=lambda x: x.stat().st_mtime, reverse=True)    return arquivosdef selecionar_arquivo_excel(diretorio: str, pattern: str, caminho_padrao: str = None) -> str:    """Seleciona arquivo Excel com interação do usuário."""    if caminho_padrao and Path(caminho_padrao).exists():        print(f"✅ Arquivo encontrado: {caminho_padrao}")        return caminho_padrao        arquivos = encontrar_arquivos_excel(diretorio, pattern)        if not arquivos:        raise FileNotFoundError(f"Nenhum arquivo Excel encontrado em {diretorio}")        if len(arquivos) == 1:        print(f"✅ Único arquivo encontrado: {arquivos[0].name}")        return str(arquivos[0])        print(f"\n📂 Encontrados {len(arquivos)} arquivos Excel:")    for i, arq in enumerate(arquivos):        mtime = pd.Timestamp.fromtimestamp(arq.stat().st_mtime).strftime("%Y-%m-%d %H:%M")        print(f"   [{i}] {arq.name} (modificado: {mtime})")        try:        escolha = input("Escolha (ENTER=mais recente): ").strip()        idx = 0 if escolha == "" else int(escolha)    except:        idx = 0        return str(arquivos[max(0, min(idx, len(arquivos)-1))])print("✅ Funções de arquivo carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - LEITURA E PROCESSAMENTO DE DADOS# =============================================================================def normalizar_nome(nome: str) -> str:    """Normaliza nome de ativo para comparação."""    if pd.isna(nome):        return ""    nome = str(nome).lower().strip()    acentos = {'á':'a','à':'a','ã':'a','â':'a','é':'e','ê':'e','í':'i',               'ó':'o','ô':'o','õ':'o','ú':'u','ç':'c'}    for k, v in acentos.items():        nome = nome.replace(k, v)    nome = re.sub(r'[-/_]', ' ', nome)    nome = re.sub(r'[^a-z0-9\s]', ' ', nome)    nome = re.sub(r'\s+', ' ', nome).strip()    return nomedef similaridade(a: str, b: str) -> float:    """Retorna score de similaridade entre 0 e 1."""    na, nb = normalizar_nome(a), normalizar_nome(b)    score = SequenceMatcher(None, na, nb).ratio()        if na in nb or nb in na:        score = max(score, 0.85)        words_a = set(na.split())    words_b = set(nb.split())    common = words_a & words_b    if len(common) >= 2:        score = max(score, 0.75 + 0.05 * len(common))        return min(score, 1.0)def parse_percentual(valor) -> float:    """Converte valor percentual para float decimal."""    if pd.isna(valor) or valor == '-' or valor == '':        return np.nan    if isinstance(valor, (int, float)):        if abs(valor) > 1:            return valor / 100        return valor    valor = str(valor).strip().replace('%', '').replace(',', '.')    try:        return float(valor) / 100    except:        return np.nandef ler_retornos_aba_excel(xlsx_path: str, sheet_name: str) -> pd.Series:    """Lê retornos mensais de uma aba do Excel."""    df = pd.read_excel(xlsx_path, sheet_name=sheet_name, header=None, engine='openpyxl')        header_row = None    for i, row in df.iterrows():        if str(row.iloc[0]).strip().lower() == 'ano':            header_row = i            break        if header_row is None:        raise ValueError(f"Cabeçalho 'Ano' não encontrado na aba '{sheet_name}'")        meses_map = {'jan':1,'fev':2,'mar':3,'abr':4,'mai':5,'jun':6,                 'jul':7,'ago':8,'set':9,'out':10,'nov':11,'dez':12}        header = [str(c).strip().lower()[:3] for c in df.iloc[header_row].values]        retornos = []    for i in range(header_row + 1, len(df)):        row = df.iloc[i]        ano_val = row.iloc[0]                if pd.isna(ano_val) or str(ano_val).strip().lower() in ['', 'estatísticas', 'estatisticas', 'meses']:            break                try:            ano = int(ano_val)        except:            break                for col_idx, col_name in enumerate(header[1:], start=1):            if col_name in meses_map:                mes = meses_map[col_name]                val = parse_percentual(row.iloc[col_idx])                if not pd.isna(val):                    data = pd.Timestamp(year=ano, month=mes, day=1)                    retornos.append({'data': data, 'retorno': val})        if not retornos:        return pd.Series(dtype=float)        df_ret = pd.DataFrame(retornos).set_index('data').sort_index()    return df_ret['retorno']def ler_pesos_resumo_robusto(xlsx_path: str) -> Tuple[Dict[str, float], str]:    """Lê pesos da aba 'Resumo' com busca robusta da coluna."""    try:        df = pd.read_excel(xlsx_path, sheet_name='Resumo', header=None, engine='openpyxl')    except Exception as e:        return {}, f"erro: {str(e)[:50]}"        header_row = None    col_ativo = None    col_peso = None        keywords_peso = ['peso', 'weight', 'alocacao', 'alocação']    keywords_carteira = ['cart', 'portfolio', 'atual', 'current']        for i, row in df.iterrows():        for j, val in enumerate(row):            if pd.isna(val):                continue            val_str = str(val).strip().lower()                        if val_str == 'ativo' or val_str == 'asset':                col_ativo = j                        has_peso = any(kw in val_str for kw in keywords_peso)            has_cart = any(kw in val_str for kw in keywords_carteira)            if has_peso and has_cart:                col_peso = j                if col_ativo is not None and col_peso is not None:            header_row = i            break        if col_peso is None:        return {}, "nao_encontrado"        print(f"   ✅ Coluna de pesos: '{df.iloc[header_row, col_peso]}' (linha {header_row})")        pesos = {}    for i in range(header_row + 1, len(df)):        row = df.iloc[i]        ativo = row.iloc[col_ativo]        peso = row.iloc[col_peso]                if pd.isna(ativo) or str(ativo).strip() == '':            continue                ativo = str(ativo).strip()        try:            peso_val = float(peso) if not pd.isna(peso) else 0.0            if peso_val > 1:                peso_val = peso_val / 100            pesos[ativo] = peso_val        except:            pesos[ativo] = 0.0        return pesos, "encontrado"print("✅ Funções de leitura carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - MAPEAMENTO DE PESOS (CORRIGIDO v2.5)# =============================================================================def mapear_pesos_para_abas(    pesos_resumo: Dict[str, float],    abas_candidatas: List[str],    abas_excluidas_min_obs: List[str],    threshold_ok: float = 0.90,    threshold_revisar: float = 0.75) -> pd.DataFrame:    """    Mapeia ativos do Resumo para abas do Excel ANTES de aplicar MIN_OBS.        Retorna DataFrame com colunas:    - ativo_resumo: nome no Resumo    - peso: peso do ativo    - aba_mapeada: nome da aba correspondente (ou None)    - score: score de similaridade    - status: OK / REVISAR / NAO_MAPEADO / MAPEADO_EXCLUIDO_MIN_OBS    - motivo: explicação do status    """    resultados = []        # Conjunto de todas as abas (incluindo as que serão excluídas por MIN_OBS)    todas_abas = set(abas_candidatas) | set(abas_excluidas_min_obs)    abas_excluidas_set = set(abas_excluidas_min_obs)        for ativo_resumo, peso in pesos_resumo.items():        if peso == 0:            continue                # Tentar match exato primeiro        if ativo_resumo in todas_abas:            if ativo_resumo in abas_excluidas_set:                status = "MAPEADO_EXCLUIDO_MIN_OBS"                motivo = f"Aba '{ativo_resumo}' excluída por MIN_OBS"                aba = ativo_resumo            else:                status = "OK"                motivo = "Match exato"                aba = ativo_resumo                        resultados.append({                'ativo_resumo': ativo_resumo,                'peso': peso,                'aba_mapeada': aba,                'score': 1.0,                'status': status,                'motivo': motivo            })            continue                # Fuzzy match - procurar em TODAS as abas (não só as que passaram MIN_OBS)        melhor_match = None        melhor_score = 0        for aba in todas_abas:            score = similaridade(ativo_resumo, aba)            if score > melhor_score:                melhor_score = score                melhor_match = aba                # Classificar o resultado        if melhor_score >= threshold_ok:            if melhor_match in abas_excluidas_set:                status = "MAPEADO_EXCLUIDO_MIN_OBS"                motivo = f"Aba '{melhor_match}' excluída por MIN_OBS"            else:                status = "OK"                motivo = f"Fuzzy match (score={melhor_score:.2f})"        elif melhor_score >= threshold_revisar:            if melhor_match in abas_excluidas_set:                status = "MAPEADO_EXCLUIDO_MIN_OBS"                motivo = f"Aba '{melhor_match}' excluída por MIN_OBS (revisar)"            else:                status = "REVISAR"                motivo = f"Score médio ({melhor_score:.2f}), verificar manualmente"        else:            status = "NAO_MAPEADO"            melhor_match = None            motivo = f"Melhor score ({melhor_score:.2f}) abaixo do threshold"                resultados.append({            'ativo_resumo': ativo_resumo,            'peso': peso,            'aba_mapeada': melhor_match,            'score': melhor_score,            'status': status,            'motivo': motivo        })        return pd.DataFrame(resultados)def gerar_relatorio_mapeamento(df_mapa: pd.DataFrame) -> Dict:    """Gera resumo numérico do mapeamento."""    peso_total = df_mapa['peso'].sum()        peso_ok = df_mapa[df_mapa['status'] == 'OK']['peso'].sum()    peso_revisar = df_mapa[df_mapa['status'] == 'REVISAR']['peso'].sum()    peso_excluido_min_obs = df_mapa[df_mapa['status'] == 'MAPEADO_EXCLUIDO_MIN_OBS']['peso'].sum()    peso_nao_mapeado = df_mapa[df_mapa['status'] == 'NAO_MAPEADO']['peso'].sum()        peso_no_universo = peso_ok + peso_revisar        return {        'peso_total_resumo': peso_total,        'peso_mapeado_ok': peso_ok,        'peso_mapeado_revisar': peso_revisar,        'peso_mapeado_no_universo': peso_no_universo,        'peso_mapeado_excluido_min_obs': peso_excluido_min_obs,        'peso_nao_mapeado': peso_nao_mapeado,        'n_ativos_ok': (df_mapa['status'] == 'OK').sum(),        'n_ativos_revisar': (df_mapa['status'] == 'REVISAR').sum(),        'n_ativos_excluidos': (df_mapa['status'] == 'MAPEADO_EXCLUIDO_MIN_OBS').sum(),        'n_ativos_nao_mapeados': (df_mapa['status'] == 'NAO_MAPEADO').sum()    }print("✅ Funções de mapeamento carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - CARREGAMENTO EM 2 ETAPAS (v2.5)# =============================================================================def carregar_metadados_excel(xlsx_path: str, ativos_excluir: List[str] = None):    """    ETAPA 1: Carrega lista de abas e contagem de meses (sem filtrar por MIN_OBS ainda).    """    if ativos_excluir is None:        ativos_excluir = []        xlsx = pd.ExcelFile(xlsx_path, engine='openpyxl')    abas = xlsx.sheet_names        metadados = {        'total_abas': len(abas),        'abas_candidatas': [],        'meses_por_ativo': {},        'ativos_excluidos_manual': [],        'erros_leitura': []    }        for aba in abas:        if aba.lower() == 'resumo':            continue                if aba.upper() in [a.upper() for a in ativos_excluir]:            metadados['ativos_excluidos_manual'].append(aba)            continue                try:            serie = ler_retornos_aba_excel(xlsx_path, aba)            n_obs = len(serie.dropna())            metadados['meses_por_ativo'][aba] = n_obs            metadados['abas_candidatas'].append(aba)        except Exception as e:            metadados['erros_leitura'].append((aba, str(e)[:50]))        return metadadosdef filtrar_por_min_obs(metadados: Dict, min_obs: int) -> Tuple[List[str], List[str]]:    """    ETAPA 2: Separa abas que passam e que não passam no filtro MIN_OBS.    """    abas_ok = []    abas_excluidas = []        for aba, n_meses in metadados['meses_por_ativo'].items():        if n_meses >= min_obs:            abas_ok.append(aba)        else:            abas_excluidas.append(aba)        return abas_ok, abas_excluidasdef carregar_retornos_das_abas(xlsx_path: str, abas: List[str]) -> pd.DataFrame:    """    ETAPA 3: Carrega retornos apenas das abas que passaram nos filtros.    """    series_list = {}    for aba in abas:        serie = ler_retornos_aba_excel(xlsx_path, aba)        series_list[aba] = serie        return pd.DataFrame(series_list)print("✅ Funções de carregamento em etapas carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - ESTATÍSTICAS E MATRIZ DE COVARIÂNCIA# =============================================================================def retorno_geometrico_anual(retornos_mensais: pd.Series) -> float:    ret = retornos_mensais.dropna()    if len(ret) == 0:        return np.nan    prod = np.prod(1 + ret)    return float(prod ** (12 / len(ret)) - 1)def retorno_aritmetico_anual(retornos_mensais: pd.Series) -> float:    return float(retornos_mensais.mean() * 12)def volatilidade_anual(retornos_mensais: pd.Series) -> float:    return float(retornos_mensais.std() * np.sqrt(12))def calcular_matriz_overlap(retornos_df: pd.DataFrame) -> pd.DataFrame:    """Calcula matriz de overlaps (meses em comum) entre pares de ativos."""    n = len(retornos_df.columns)    ativos = retornos_df.columns    overlap = pd.DataFrame(index=ativos, columns=ativos, dtype=float)        for i, ativo_i in enumerate(ativos):        for j, ativo_j in enumerate(ativos):            if i == j:                overlap.iloc[i, j] = len(retornos_df[ativo_i].dropna())            else:                mask = retornos_df[ativo_i].notna() & retornos_df[ativo_j].notna()                overlap.iloc[i, j] = mask.sum()        return overlapdef aplicar_min_overlap_covariancia(cov_matrix: pd.DataFrame, overlap_matrix: pd.DataFrame,                                     min_overlap: int) -> Tuple[pd.DataFrame, int]:    """    Zera covariâncias de pares com overlap < MIN_OVERLAP.    Retorna matriz ajustada e número de pares afetados.    """    cov_ajustada = cov_matrix.copy()    pares_afetados = 0        for i, ativo_i in enumerate(cov_matrix.index):        for j, ativo_j in enumerate(cov_matrix.columns):            if i != j:                if overlap_matrix.iloc[i, j] < min_overlap:                    cov_ajustada.iloc[i, j] = 0                    cov_ajustada.iloc[j, i] = 0                    pares_afetados += 1        return cov_ajustada, pares_afetados // 2  # Dividir por 2 porque contou 2xdef aplicar_vol_floor_covariancia(cov_matrix: pd.DataFrame, vol_floor_mensal: float) -> pd.DataFrame:    cov_floor_mensal = vol_floor_mensal ** 2    cov_floor_anual = cov_floor_mensal * 12        cov_ajustada = cov_matrix.copy()    ativos_ajustados = []        for i, ativo in enumerate(cov_matrix.index):        var_atual = cov_matrix.iloc[i, i]        if var_atual < cov_floor_anual:            cov_ajustada.iloc[i, i] = cov_floor_anual            ativos_ajustados.append(ativo)        return cov_ajustada, ativos_ajustadosdef corrigir_matriz_psd(cov_matrix: pd.DataFrame, epsilon: float = 1e-8) -> Tuple[pd.DataFrame, bool]:    arr = cov_matrix.values    eigenvalues, eigenvectors = eigh(arr)        if np.min(eigenvalues) >= -epsilon:        return cov_matrix, False        eigenvalues_corrigidos = np.maximum(eigenvalues, epsilon)    arr_corrigido = eigenvectors @ np.diag(eigenvalues_corrigidos) @ eigenvectors.T    arr_corrigido = (arr_corrigido + arr_corrigido.T) / 2        return pd.DataFrame(arr_corrigido, index=cov_matrix.index, columns=cov_matrix.columns), Truedef calcular_estatisticas_portfolio(retornos_df: pd.DataFrame, usar_geometrico: bool = True,                                    vol_floor_mensal: float = 0.0, min_overlap: int = 0):    """Calcula retornos esperados e matriz de covariância com ajustes."""    if usar_geometrico:        retornos_esperados = retornos_df.apply(retorno_geometrico_anual)    else:        retornos_esperados = retornos_df.apply(retorno_aritmetico_anual)        cov_matrix = retornos_df.cov() * 12        # Aplicar MIN_OVERLAP    pares_zerados = 0    if min_overlap > 0:        overlap_matrix = calcular_matriz_overlap(retornos_df)        cov_matrix, pares_zerados = aplicar_min_overlap_covariancia(cov_matrix, overlap_matrix, min_overlap)        # Aplicar VOL_FLOOR    ativos_floor = []    if vol_floor_mensal > 0:        cov_matrix, ativos_floor = aplicar_vol_floor_covariancia(cov_matrix, vol_floor_mensal)        return retornos_esperados, cov_matrix, pares_zerados, ativos_floorprint("✅ Funções de estatística carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - OTIMIZAÇÃO (COM CHECAGEM DE SUCESSO + FALLBACK)# =============================================================================class ResultadoOtimizacao:    """Classe para armazenar resultado da otimização com metadados."""    def __init__(self, pesos, sucesso, mensagem, metodo):        self.pesos = pesos        self.sucesso = sucesso        self.mensagem = mensagem        self.metodo = metododef calcular_metricas_portfolio(pesos, retornos_esperados, cov_matrix, rf_anual=0.0):    pesos = np.array(pesos, dtype=float)    retorno = float(np.dot(pesos, retornos_esperados))    volatilidade = float(np.sqrt(np.dot(pesos.T, np.dot(cov_matrix, pesos))))        if volatilidade > 1e-8:        sharpe = (retorno - rf_anual) / volatilidade    else:        sharpe = np.nan        return retorno, volatilidade, sharpedef pesos_equal_weight(n: int) -> np.ndarray:    """Fallback: pesos iguais para todos os ativos."""    return np.ones(n) / ndef otimizar_min_volatilidade(retornos_esperados, cov_matrix, peso_min=0.0, peso_max=1.0) -> ResultadoOtimizacao:    """Encontra carteira GMV com checagem de sucesso."""    n = len(retornos_esperados)    pesos_iniciais = np.ones(n) / n    limites = tuple((peso_min, peso_max) for _ in range(n))    restricoes = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0}        def objetivo(w):        return np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))        result = minimize(objetivo, pesos_iniciais, method='SLSQP', bounds=limites,                      constraints=restricoes, options={'maxiter': 1000})        if result.success:        return ResultadoOtimizacao(result.x, True, "Convergiu", "GMV_SLSQP")    else:        print(f"   ⚠️ GMV não convergiu: {result.message}")        print(f"      Usando fallback: equal-weight")        return ResultadoOtimizacao(pesos_equal_weight(n), False, result.message, "GMV_FALLBACK_EW")def otimizar_max_sharpe(retornos_esperados, cov_matrix, rf_anual=0.0, peso_min=0.0, peso_max=1.0) -> ResultadoOtimizacao:    """Encontra carteira de máximo Sharpe com checagem de sucesso."""    n = len(retornos_esperados)    pesos_iniciais = np.ones(n) / n    limites = tuple((peso_min, peso_max) for _ in range(n))    restricoes = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0}        def objetivo(w):        ret = np.dot(w, retornos_esperados)        vol = np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))        if vol < 1e-8:            return 1e10        return -(ret - rf_anual) / vol        result = minimize(objetivo, pesos_iniciais, method='SLSQP', bounds=limites,                      constraints=restricoes, options={'maxiter': 1000})        if result.success:        return ResultadoOtimizacao(result.x, True, "Convergiu", "MAX_SHARPE_SLSQP")    else:        print(f"   ⚠️ Max Sharpe não convergiu: {result.message}")        print(f"      Usando fallback: equal-weight")        return ResultadoOtimizacao(pesos_equal_weight(n), False, result.message, "MAX_SHARPE_FALLBACK_EW")def otimizar_para_retorno_alvo(retornos_esperados, cov_matrix, retorno_alvo, peso_min=0.0, peso_max=1.0):    """Encontra carteira de mínima variância para um retorno-alvo."""    n = len(retornos_esperados)    pesos_iniciais = np.ones(n) / n    limites = tuple((peso_min, peso_max) for _ in range(n))    restricoes = [        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},        {'type': 'eq', 'fun': lambda x: np.dot(x, retornos_esperados) - retorno_alvo}    ]        def objetivo(w):        return np.dot(w.T, np.dot(cov_matrix, w))        result = minimize(objetivo, pesos_iniciais, method='SLSQP', bounds=limites,                      constraints=restricoes, options={'maxiter': 500})        return result.x if result.success else Nonedef calcular_fronteira_eficiente(retornos_esperados, cov_matrix, rf_anual, n_pontos=50,                                  peso_min=0.0, peso_max=1.0):    """Calcula a fronteira eficiente por otimização."""    result_gmv = otimizar_min_volatilidade(retornos_esperados, cov_matrix, peso_min, peso_max)    ret_gmv, vol_gmv, _ = calcular_metricas_portfolio(result_gmv.pesos, retornos_esperados, cov_matrix, rf_anual)        result_max_sharpe = otimizar_max_sharpe(retornos_esperados, cov_matrix, rf_anual, peso_min, peso_max)    ret_max_sharpe, vol_max_sharpe, _ = calcular_metricas_portfolio(result_max_sharpe.pesos, retornos_esperados, cov_matrix, rf_anual)        ret_max = np.max(retornos_esperados)    retornos_alvo = np.linspace(ret_gmv, min(ret_max, ret_max_sharpe * 1.5), n_pontos)        fronteira_vols = []    fronteira_rets = []        for ret_alvo in retornos_alvo:        pesos = otimizar_para_retorno_alvo(retornos_esperados, cov_matrix, ret_alvo, peso_min, peso_max)        if pesos is not None:            ret, vol, _ = calcular_metricas_portfolio(pesos, retornos_esperados, cov_matrix, rf_anual)            fronteira_vols.append(vol)            fronteira_rets.append(ret)        pontos = sorted(set(zip(fronteira_vols, fronteira_rets)), key=lambda x: x[0])        if pontos:        fronteira_vols, fronteira_rets = zip(*pontos)        return np.array(fronteira_vols), np.array(fronteira_rets)    else:        return np.array([vol_gmv]), np.array([ret_gmv])print("✅ Funções de otimização carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - MONTE CARLO# =============================================================================def simular_portfolios_monte_carlo(    retornos_esperados: np.ndarray,     cov_matrix: np.ndarray,     rf_anual: float,    n_portfolios: int = 50000,    seed: int = 42,    alpha_conc: float = 0.2,    alpha_div: float = 1.0,    frac_cantos: float = 0.05,    frac_sparse: float = 0.15) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:    """Monte Carlo com cantos, sparse e mix concentrado/diversificado."""    np.random.seed(seed)    n_ativos = len(retornos_esperados)        n_cantos = int(n_portfolios * frac_cantos)    n_sparse = int(n_portfolios * frac_sparse)    n_conc = int((n_portfolios - n_cantos - n_sparse) * 0.6)    n_div = n_portfolios - n_cantos - n_sparse - n_conc        pesos_list = []        # Cantos    for _ in range(n_cantos):        w = np.zeros(n_ativos)        idx_principal = np.random.randint(0, n_ativos)        peso_principal = np.random.uniform(0.85, 0.98)        w[idx_principal] = peso_principal        resto = 1.0 - peso_principal        outros_idx = [i for i in range(n_ativos) if i != idx_principal]        if outros_idx:            w_resto = np.random.dirichlet([0.5] * len(outros_idx)) * resto            for i, idx in enumerate(outros_idx):                w[idx] = w_resto[i]        pesos_list.append(w)        # Sparse    for _ in range(n_sparse):        w = np.zeros(n_ativos)        k = np.random.randint(3, min(9, n_ativos + 1))        idx_selecionados = np.random.choice(n_ativos, size=k, replace=False)        w_k = np.random.dirichlet([0.5] * k)        w[idx_selecionados] = w_k        pesos_list.append(w)        # Concentrado    w_conc = np.random.dirichlet([alpha_conc] * n_ativos, size=n_conc)    pesos_list.extend(w_conc)        # Diversificado    w_div = np.random.dirichlet([alpha_div] * n_ativos, size=n_div)    pesos_list.extend(w_div)        pesos = np.array(pesos_list)        retornos = pesos @ retornos_esperados    volatilidades = np.sqrt(np.einsum('ij,jk,ik->i', pesos, cov_matrix, pesos))    sharpes = np.where(volatilidades > 1e-8, (retornos - rf_anual) / volatilidades, np.nan)        return retornos, volatilidades, sharpes, pesosprint("✅ Funções de Monte Carlo carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - TAXA LIVRE DE RISCO (CDI)# =============================================================================def obter_cdi_atual():    try:        url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados/ultimos/1?formato=json"        df = pd.read_json(url)        if df.empty:            return None        valor_str = str(df.loc[0, "valor"]).replace(",", ".")        taxa_diaria = float(valor_str) / 100.0        return float((1.0 + taxa_diaria) ** 252 - 1.0)    except:        return Nonedef obter_cdi_periodo(data_inicio: str, data_fim: str):    try:        di = pd.to_datetime(data_inicio).strftime("%d/%m/%Y")        df_str = pd.to_datetime(data_fim).strftime("%d/%m/%Y")        url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados?formato=json&dataInicial={di}&dataFinal={df_str}"                cdi = pd.read_json(url)        if cdi.empty:            return None                cdi["valor"] = cdi["valor"].astype(str).str.replace(",", ".", regex=False)        cdi["valor"] = pd.to_numeric(cdi["valor"], errors="coerce") / 100.0                return float(np.expm1(np.log1p(cdi["valor"].dropna()).mean() * 252))    except:        return Nonedef obter_taxa_livre_risco(modo, fallback, manual=None, data_inicio=None, data_fim=None):    if modo == "manual":        return manual if manual is not None else fallback        if modo == "atual":        rf = obter_cdi_atual()        return rf if rf is not None else fallback        if modo == "media_periodo" and data_inicio and data_fim:        rf = obter_cdi_periodo(data_inicio, data_fim)        if rf is not None:            return rf        rf = obter_cdi_atual()        return rf if rf is not None else fallback        return fallbackprint("✅ Funções de RF carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - VISUALIZAÇÃO# =============================================================================def criar_grafico_fronteira(ret_sim, vol_sim, sharpe_sim, fronteira_vols, fronteira_rets,                            carteiras_especiais, rf_anual, titulo="Fronteira Eficiente"):    fig = go.Figure()        # Nuvem    fig.add_trace(go.Scatter(        x=vol_sim * 100, y=ret_sim * 100,        mode='markers',        marker=dict(size=2, color=sharpe_sim, colorscale='Viridis',                    colorbar=dict(title='Sharpe', x=1.02), opacity=0.4),        name='Simulados',        hovertemplate='Vol: %{x:.2f}%<br>Ret: %{y:.2f}%<extra></extra>'    ))        # Fronteira    fig.add_trace(go.Scatter(        x=fronteira_vols * 100, y=fronteira_rets * 100,        mode='lines', line=dict(color='red', width=3),        name='Fronteira Eficiente'    ))        # Carteiras especiais    estilos = {        'GMV': {'color': 'blue', 'symbol': 'diamond', 'size': 18},        'Max Sharpe': {'color': 'green', 'symbol': 'triangle-up', 'size': 18},        'Carteira Atual': {'color': 'gold', 'symbol': 'star', 'size': 24}    }        for nome, dados in carteiras_especiais.items():        estilo = estilos.get(nome, {'color': 'purple', 'symbol': 'circle', 'size': 15})                pesos = dados.get('pesos', [])        nomes_ativos = dados.get('nomes', [])        if len(pesos) > 0 and len(nomes_ativos) > 0:            idx_sorted = np.argsort(pesos)[::-1]            top_pesos = '<br>'.join([f"{nomes_ativos[i]}: {pesos[i]*100:.1f}%"                                      for i in idx_sorted[:10] if pesos[i] > 0.01])        else:            top_pesos = ''                hover = f"<b>{nome}</b><br>Ret: {dados['retorno']*100:.2f}%<br>Vol: {dados['volatilidade']*100:.2f}%<br>Sharpe: {dados['sharpe']:.3f}<br><br>{top_pesos}"                fig.add_trace(go.Scatter(            x=[dados['volatilidade'] * 100], y=[dados['retorno'] * 100],            mode='markers+text',            marker=dict(size=estilo['size'], color=estilo['color'], symbol=estilo['symbol'],                        line=dict(width=2, color='black')),            text=[nome], textposition='top center', name=nome,            hovertemplate=hover + '<extra></extra>'        ))        fig.add_hline(y=rf_anual * 100, line_dash="dash", line_color="gray",                  annotation_text=f"RF: {rf_anual*100:.2f}%")        fig.update_layout(        title=dict(text=titulo, x=0.5),        xaxis_title="Volatilidade (%)", yaxis_title="Retorno (%)",        template="plotly_white", width=1100, height=700,        legend=dict(x=0.02, y=0.98)    )        return figdef criar_heatmap_correlacao(corr_matrix, titulo="Correlação"):    fig = go.Figure(data=go.Heatmap(        z=corr_matrix.values,        x=list(corr_matrix.columns), y=list(corr_matrix.index),        colorscale='RdBu', zmid=0, zmin=-1, zmax=1,        hovertemplate='%{x} vs %{y}<br>Corr: %{z:.3f}<extra></extra>'    ))    fig.update_layout(title=dict(text=titulo, x=0.5), width=900, height=800, xaxis=dict(side='top'))    return figdef criar_grafico_barras_pesos(carteiras: Dict, nomes_ativos: List[str], top_n: int = 15):    fig = go.Figure()    cores = {'GMV': '#636EFA', 'Max Sharpe': '#00CC96', 'Carteira Atual': '#FFA15A'}        for nome_cart, dados in carteiras.items():        pesos = np.array(dados.get('pesos', []))        if len(pesos) == 0:            continue                df_temp = pd.DataFrame({'ativo': nomes_ativos, 'peso': pesos})        df_temp = df_temp.nlargest(top_n, 'peso')                fig.add_trace(go.Bar(            name=nome_cart, x=df_temp['ativo'], y=df_temp['peso'] * 100,            marker_color=cores.get(nome_cart, '#AB63FA')        ))        fig.update_layout(        title="Comparativo de Pesos - Top Ativos",        xaxis_title="Ativo", yaxis_title="Peso (%)",        barmode='group', template="plotly_white", width=1000, height=500    )        return figprint("✅ Funções de visualização carregadas!")

---## 📥 BLOCO 2 - CARREGAMENTO DE DADOS (CORRIGIDO v2.5)**Fluxo corrigido:**1. Carregar metadados de TODAS as abas (antes de filtrar)2. Ler pesos do Resumo3. Fazer mapeamento ANTES do filtro MIN_OBS4. Aplicar MIN_OBS e gerar relatório de mapeamento5. Carregar retornos apenas das abas que passaram

In [ ]:
# =============================================================================# ETAPA 1: SELECIONAR ARQUIVO EXCEL# =============================================================================print("=" * 60)print("CARREGAMENTO DE DADOS (v2.5)")print("=" * 60)caminho_padrao = f"{EXCEL_DIR}/Rendimentos_Mensais_Ativos_v7.0.xlsx"EXCEL_PATH = selecionar_arquivo_excel(EXCEL_DIR, EXCEL_PATTERN, caminho_padrao)

In [ ]:
# =============================================================================# ETAPA 2: CARREGAR METADADOS (ANTES DE FILTRAR MIN_OBS)# =============================================================================print("\n📂 Carregando metadados das abas...")metadados = carregar_metadados_excel(EXCEL_PATH, ativos_excluir=ATIVOS_EXCLUIR)print(f"\n   Total de abas candidatas: {len(metadados['abas_candidatas'])}")print(f"   Excluídos manualmente: {metadados['ativos_excluidos_manual']}")if metadados['erros_leitura']:    print(f"   Erros de leitura: {len(metadados['erros_leitura'])}")

In [ ]:
# =============================================================================# ETAPA 3: LER PESOS DO RESUMO# =============================================================================print("\n📊 Buscando coluna de pesos...")pesos_resumo, status_pesos = ler_pesos_resumo_robusto(EXCEL_PATH)if status_pesos == "nao_encontrado":    print("   ⚠️ ATENÇÃO: Coluna de pesos não encontrada!")elif status_pesos.startswith("erro"):    print(f"   ❌ Erro: {status_pesos}")else:    print(f"   Total de ativos com peso > 0: {sum(1 for v in pesos_resumo.values() if v > 0)}")    print(f"   Soma dos pesos: {sum(pesos_resumo.values()):.2%}")

In [ ]:
# =============================================================================# ETAPA 4: SEPARAR ABAS POR MIN_OBS (ANTES DO MAPEAMENTO!)# =============================================================================print("\n📊 Aplicando filtro MIN_OBS...")abas_ok, abas_excluidas_min_obs = filtrar_por_min_obs(metadados, MIN_OBS)print(f"   Abas que passaram: {len(abas_ok)}")print(f"   Abas excluídas por MIN_OBS: {len(abas_excluidas_min_obs)}")if abas_excluidas_min_obs:    print("\n   Detalhes das abas excluídas por MIN_OBS:")    for aba in abas_excluidas_min_obs[:10]:        n_meses = metadados['meses_por_ativo'].get(aba, 0)        print(f"      - {aba}: {n_meses} meses < {MIN_OBS}")

In [ ]:
# =============================================================================# ETAPA 5: MAPEAMENTO DE PESOS (USANDO TODAS AS ABAS!)# =============================================================================print("\n" + "=" * 60)print("MAPEAMENTO DE PESOS (CORRIGIDO v2.5)")print("=" * 60)if pesos_resumo and status_pesos == "encontrado":    df_mapa = mapear_pesos_para_abas(        pesos_resumo=pesos_resumo,        abas_candidatas=abas_ok,        abas_excluidas_min_obs=abas_excluidas_min_obs,        threshold_ok=THRESHOLD_OK,        threshold_revisar=THRESHOLD_REVISAR    )        # Gerar relatório    relatorio_mapa = gerar_relatorio_mapeamento(df_mapa)        print(f"\n📊 RESUMO DO MAPEAMENTO:")    print(f"   Peso total no Resumo: {relatorio_mapa['peso_total_resumo']:.2%}")    print(f"   Peso mapeado OK: {relatorio_mapa['peso_mapeado_ok']:.2%} ({relatorio_mapa['n_ativos_ok']} ativos)")    print(f"   Peso mapeado REVISAR: {relatorio_mapa['peso_mapeado_revisar']:.2%} ({relatorio_mapa['n_ativos_revisar']} ativos)")    print(f"   Peso no universo: {relatorio_mapa['peso_mapeado_no_universo']:.2%}")    print(f"   ⚠️  Peso MAPEADO_EXCLUIDO_MIN_OBS: {relatorio_mapa['peso_mapeado_excluido_min_obs']:.2%} ({relatorio_mapa['n_ativos_excluidos']} ativos)")    print(f"   ❌ Peso NÃO MAPEADO: {relatorio_mapa['peso_nao_mapeado']:.2%} ({relatorio_mapa['n_ativos_nao_mapeados']} ativos)")        # Mostrar DataFrame de mapeamento    print("\n📋 TABELA DE MAPEAMENTO:")    display(df_mapa.sort_values('peso', ascending=False).head(25))        TEM_PESOS = Trueelse:    df_mapa = None    relatorio_mapa = None    TEM_PESOS = False    print("\n⚠️ Pesos não disponíveis - continuando sem carteira atual")

In [ ]:
# =============================================================================# ETAPA 6: CARREGAR RETORNOS (APENAS ABAS QUE PASSARAM)# =============================================================================print("\n📂 Carregando retornos das abas que passaram no filtro...")retornos_df = carregar_retornos_das_abas(EXCEL_PATH, abas_ok)ATIVOS_OTIMIZACAO = list(retornos_df.columns)NUM_ATIVOS = len(ATIVOS_OTIMIZACAO)print(f"\n✅ Dados carregados!")print(f"   Ativos no universo de otimização: {NUM_ATIVOS}")print(f"   Período: {retornos_df.index.min().strftime('%Y-%m')} a {retornos_df.index.max().strftime('%Y-%m')}")print(f"   Total de meses: {len(retornos_df)}")

In [ ]:
# =============================================================================# ETAPA 7: CRIAR VETOR DE PESOS DA CARTEIRA ATUAL# =============================================================================if TEM_PESOS and df_mapa is not None:    print("\n📊 Criando vetor de pesos da carteira atual...")        # Filtrar apenas ativos que estão no universo de otimização    df_no_universo = df_mapa[df_mapa['status'].isin(['OK', 'REVISAR'])].copy()        pesos_carteira = np.zeros(NUM_ATIVOS)    for _, row in df_no_universo.iterrows():        aba = row['aba_mapeada']        if aba in ATIVOS_OTIMIZACAO:            idx = ATIVOS_OTIMIZACAO.index(aba)            pesos_carteira[idx] += row['peso']        soma_pesos = pesos_carteira.sum()    print(f"   Soma dos pesos mapeados no universo: {soma_pesos:.2%}")        if soma_pesos > 0:        if abs(soma_pesos - 1.0) > 0.01:            print(f"   Renormalizando para 100% (base: {soma_pesos:.2%})")            pesos_carteira = pesos_carteira / soma_pesos        TEM_CARTEIRA_ATUAL = True    else:        TEM_CARTEIRA_ATUAL = False        print("   ⚠️ Nenhum peso no universo - sem carteira atual")else:    pesos_carteira = None    TEM_CARTEIRA_ATUAL = False

---## 📈 BLOCO 3 - CÁLCULO DE ESTATÍSTICAS

In [ ]:
# =============================================================================# TAXA LIVRE DE RISCO# =============================================================================print("\n" + "=" * 60)print("TAXA LIVRE DE RISCO (CDI)")print("=" * 60)periodo_inicio = retornos_df.index.min().strftime('%Y-%m-%d')periodo_fim = retornos_df.index.max().strftime('%Y-%m-%d')RF_ANUAL = obter_taxa_livre_risco(RF_MODO, RF_FALLBACK, RF_MANUAL, periodo_inicio, periodo_fim)print(f"\n📊 RF anual (CDI): {RF_ANUAL*100:.2f}% a.a.")print(f"   Modo: {RF_MODO}")

In [ ]:
# =============================================================================# ESTATÍSTICAS DOS ATIVOS (COM MIN_OVERLAP)# =============================================================================print("\n" + "=" * 60)print("ESTATÍSTICAS DOS ATIVOS")print("=" * 60)retornos_esperados, cov_matrix, pares_zerados, ativos_floor = calcular_estatisticas_portfolio(    retornos_df,     usar_geometrico=USAR_MEDIA_GEOMETRICA,     vol_floor_mensal=VOL_FLOOR_MENSAL,    min_overlap=MIN_OVERLAP)if pares_zerados > 0:    print(f"\n⚠️ MIN_OVERLAP aplicado: {pares_zerados} pares de covariância zerados")if ativos_floor:    print(f"⚠️ VOL_FLOOR aplicado em: {ativos_floor[:5]}{'...' if len(ativos_floor) > 5 else ''}")cov_matrix, foi_corrigida = corrigir_matriz_psd(cov_matrix)if foi_corrigida:    print("⚠️ Matriz de covariância corrigida para PSD")corr_matrix = retornos_df.corr()volatilidades = pd.Series({col: np.sqrt(cov_matrix.loc[col, col]) for col in cov_matrix.columns})print(f"\n{'Ativo':<30} {'Ret.':<10} {'Vol.':<10} {'Sharpe':<8} {'Meses':<6}")print("-" * 70)for ativo in ATIVOS_OTIMIZACAO[:15]:    ret = retornos_esperados[ativo]    vol = volatilidades[ativo]    sharpe = (ret - RF_ANUAL) / vol if vol > 1e-6 else np.nan    n_meses = metadados['meses_por_ativo'].get(ativo, 0)    print(f"{ativo:<30} {ret*100:>7.2f}% {vol*100:>7.2f}% {sharpe:>7.2f} {n_meses:>5}")if len(ATIVOS_OTIMIZACAO) > 15:    print(f"... e mais {len(ATIVOS_OTIMIZACAO) - 15} ativos")

---## ⚙️ BLOCO 4 - OTIMIZAÇÃO DE PORTFÓLIO

In [ ]:
# =============================================================================# OTIMIZAÇÃO: GMV, MAX SHARPE E FRONTEIRA# =============================================================================print("\n" + "=" * 60)print("OTIMIZAÇÃO DE PORTFÓLIO")print("=" * 60)ret_array = retornos_esperados.valuescov_array = cov_matrix.values# Registro de status das otimizaçõesotim_status = {}print("\n🔍 Otimizando GMV...")result_gmv = otimizar_min_volatilidade(ret_array, cov_array, PESO_MIN_ATIVO, PESO_MAX_ATIVO)pesos_gmv = result_gmv.pesosret_gmv, vol_gmv, sharpe_gmv = calcular_metricas_portfolio(pesos_gmv, ret_array, cov_array, RF_ANUAL)otim_status['GMV'] = {'sucesso': result_gmv.sucesso, 'metodo': result_gmv.metodo, 'msg': result_gmv.mensagem}print("🔍 Otimizando Máximo Sharpe...")result_max_sharpe = otimizar_max_sharpe(ret_array, cov_array, RF_ANUAL, PESO_MIN_ATIVO, PESO_MAX_ATIVO)pesos_max_sharpe = result_max_sharpe.pesosret_max_sharpe, vol_max_sharpe, sharpe_max_sharpe = calcular_metricas_portfolio(pesos_max_sharpe, ret_array, cov_array, RF_ANUAL)otim_status['Max Sharpe'] = {'sucesso': result_max_sharpe.sucesso, 'metodo': result_max_sharpe.metodo, 'msg': result_max_sharpe.mensagem}print("🔍 Calculando Fronteira Eficiente...")fronteira_vols, fronteira_rets = calcular_fronteira_eficiente(    ret_array, cov_array, RF_ANUAL,     n_pontos=N_PONTOS_FRONTEIRA,    peso_min=PESO_MIN_ATIVO, peso_max=PESO_MAX_ATIVO)print(f"   {len(fronteira_vols)} pontos calculados")if TEM_CARTEIRA_ATUAL:    ret_atual, vol_atual, sharpe_atual = calcular_metricas_portfolio(pesos_carteira, ret_array, cov_array, RF_ANUAL)# Resumo de statusprint("\n📊 STATUS DAS OTIMIZAÇÕES:")for nome, status in otim_status.items():    emoji = "✅" if status['sucesso'] else "⚠️"    print(f"   {emoji} {nome}: {status['metodo']} - {status['msg']}")

---## 🎲 BLOCO 5 - SIMULAÇÃO MONTE CARLO

In [ ]:
# =============================================================================# SIMULAÇÃO MONTE CARLO# =============================================================================print("\n" + "=" * 60)print(f"SIMULAÇÃO MONTE CARLO ({NUM_PORTFOLIOS:,} portfólios)")print("=" * 60)ret_sim, vol_sim, sharpe_sim, pesos_sim = simular_portfolios_monte_carlo(    ret_array, cov_array, RF_ANUAL,    n_portfolios=NUM_PORTFOLIOS,    seed=RANDOM_SEED,    alpha_conc=MC_ALPHA_CONC,    alpha_div=MC_ALPHA_DIV,    frac_cantos=MC_FRAC_CANTOS,    frac_sparse=MC_FRAC_SPARSE)print(f"\n✅ Simulação concluída!")print(f"   Retorno: min={ret_sim.min()*100:.2f}%, max={ret_sim.max()*100:.2f}%")print(f"   Volatilidade: min={vol_sim.min()*100:.2f}%, max={vol_sim.max()*100:.2f}%")

---## 📊 BLOCO 6 - VISUALIZAÇÕES

In [ ]:
# =============================================================================# GRÁFICO: FRONTEIRA EFICIENTE# =============================================================================carteiras_especiais = {    'GMV': {        'retorno': ret_gmv, 'volatilidade': vol_gmv, 'sharpe': sharpe_gmv,        'pesos': pesos_gmv, 'nomes': ATIVOS_OTIMIZACAO    },    'Max Sharpe': {        'retorno': ret_max_sharpe, 'volatilidade': vol_max_sharpe, 'sharpe': sharpe_max_sharpe,        'pesos': pesos_max_sharpe, 'nomes': ATIVOS_OTIMIZACAO    }}if TEM_CARTEIRA_ATUAL:    carteiras_especiais['Carteira Atual'] = {        'retorno': ret_atual, 'volatilidade': vol_atual, 'sharpe': sharpe_atual,        'pesos': pesos_carteira, 'nomes': ATIVOS_OTIMIZACAO    }fig_frontier = criar_grafico_fronteira(    ret_sim, vol_sim, sharpe_sim,    fronteira_vols, fronteira_rets,    carteiras_especiais, RF_ANUAL,    titulo="Fronteira Eficiente - PyPortfolio v2.5")fig_frontier.show()

In [ ]:
# =============================================================================# GRÁFICO: MATRIZ DE CORRELAÇÃO# =============================================================================fig_corr = criar_heatmap_correlacao(corr_matrix, "Matriz de Correlação - Retornos Mensais")fig_corr.show()

In [ ]:
# =============================================================================# GRÁFICO: COMPARATIVO DE PESOS# =============================================================================fig_pesos = criar_grafico_barras_pesos(carteiras_especiais, ATIVOS_OTIMIZACAO, top_n=TOP_N_PESOS)fig_pesos.show()

---## 📋 BLOCO 7 - TABELAS ESTRUTURADAS (NOVO v2.5)

In [ ]:
# =============================================================================# TABELA 1: MÉTRICAS DAS CARTEIRAS# =============================================================================print("\n" + "=" * 60)print("TABELA DE MÉTRICAS DAS CARTEIRAS")print("=" * 60)dados_metricas = [    {'Carteira': 'GMV', 'Retorno Anual (%)': ret_gmv * 100, 'Volatilidade Anual (%)': vol_gmv * 100, 'Sharpe': sharpe_gmv},    {'Carteira': 'Max Sharpe', 'Retorno Anual (%)': ret_max_sharpe * 100, 'Volatilidade Anual (%)': vol_max_sharpe * 100, 'Sharpe': sharpe_max_sharpe}]if TEM_CARTEIRA_ATUAL:    dados_metricas.append({        'Carteira': 'Carteira Atual',         'Retorno Anual (%)': ret_atual * 100,         'Volatilidade Anual (%)': vol_atual * 100,         'Sharpe': sharpe_atual    })df_metricas = pd.DataFrame(dados_metricas)df_metricas['Retorno Anual (%)'] = df_metricas['Retorno Anual (%)'].round(2)df_metricas['Volatilidade Anual (%)'] = df_metricas['Volatilidade Anual (%)'].round(2)df_metricas['Sharpe'] = df_metricas['Sharpe'].round(3)print("\n")display(df_metricas.set_index('Carteira'))

In [ ]:
# =============================================================================# TABELA 2: PESOS POR ATIVO (COMPARATIVO)# =============================================================================print("\n" + "=" * 60)print("TABELA DE PESOS POR ATIVO")print("=" * 60)df_pesos = pd.DataFrame({    'Ativo': ATIVOS_OTIMIZACAO,    'GMV (%)': pesos_gmv * 100,    'Max Sharpe (%)': pesos_max_sharpe * 100})if TEM_CARTEIRA_ATUAL:    df_pesos['Carteira Atual (%)'] = pesos_carteira * 100    df_pesos = df_pesos.sort_values('Carteira Atual (%)', ascending=False)else:    df_pesos = df_pesos.sort_values('Max Sharpe (%)', ascending=False)# Arredondarfor col in df_pesos.columns[1:]:    df_pesos[col] = df_pesos[col].round(2)print(f"\n📊 TOP {TOP_N_PESOS} ATIVOS:")display(df_pesos.head(TOP_N_PESOS).reset_index(drop=True))# Disponibilizar tabela completaprint(f"\n(Tabela completa disponível em `df_pesos` com {len(df_pesos)} ativos)")

---## 📋 BLOCO 8 - RESUMO FINAL

In [ ]:
# =============================================================================# RESUMO FINAL# =============================================================================print("\n" + "=" * 70)print("RESUMO DA ANÁLISE - PyPortfolio v2.5")print("=" * 70)print(f"\n📊 CONFIGURAÇÃO")print(f"   Arquivo: {Path(EXCEL_PATH).name}")print(f"   Período: {retornos_df.index.min().strftime('%Y-%m')} a {retornos_df.index.max().strftime('%Y-%m')}")print(f"   Ativos no universo: {NUM_ATIVOS}")print(f"   MIN_OBS: {MIN_OBS} | MIN_OVERLAP: {MIN_OVERLAP} | VOL_FLOOR: {VOL_FLOOR_MENSAL*100:.1f}%")print(f"   RF (CDI): {RF_ANUAL*100:.2f}% a.a.")if TEM_PESOS and relatorio_mapa:    print(f"\n📊 MAPEAMENTO DE PESOS")    print(f"   Peso no universo: {relatorio_mapa['peso_mapeado_no_universo']:.2%}")    print(f"   Peso excluído (MIN_OBS): {relatorio_mapa['peso_mapeado_excluido_min_obs']:.2%}")    print(f"   Peso não mapeado: {relatorio_mapa['peso_nao_mapeado']:.2%}")print(f"\n📈 CARTEIRAS OTIMIZADAS")print(f"   {'Carteira':<20} {'Retorno':<12} {'Vol':<12} {'Sharpe':<10} {'Status'}")print(f"   {'-'*66}")print(f"   {'GMV':<20} {ret_gmv*100:>9.2f}%  {vol_gmv*100:>9.2f}%  {sharpe_gmv:>8.3f}   {'✅' if otim_status['GMV']['sucesso'] else '⚠️ fallback'}")print(f"   {'Max Sharpe':<20} {ret_max_sharpe*100:>9.2f}%  {vol_max_sharpe*100:>9.2f}%  {sharpe_max_sharpe:>8.3f}   {'✅' if otim_status['Max Sharpe']['sucesso'] else '⚠️ fallback'}")if TEM_CARTEIRA_ATUAL:    print(f"   {'Carteira Atual ⭐':<20} {ret_atual*100:>9.2f}%  {vol_atual*100:>9.2f}%  {sharpe_atual:>8.3f}")print("\n" + "=" * 70)print("✅ Análise concluída com sucesso!")print("=" * 70)

---## 🗺️ ROADMAP - Próximas Versões (Priorizado)### Alta Prioridade (v2.6-v2.7)| # | Melhoria | Impacto | Complexidade ||---|----------|---------|--------------|| 1 | **Conversão USD→BRL (FX)** | Retornos corretos para ativos US | Média || 2 | **Fonte secundária yfinance + cache** | Robustez quando Excel não disponível | Média || 3 | **Fuzzy matching com rapidfuzz** | Melhor precisão no mapeamento | Baixa |### Média Prioridade (v2.8-v3.0)| # | Melhoria | Impacto | Complexidade ||---|----------|---------|--------------|| 4 | **Restrições por classe e por ativo** | Carteiras mais realistas | Média || 5 | **Backtesting out-of-sample** | Validação de performance | Alta || 6 | **Covariância robusta (Ledoit-Wolf)** | Menos overfitting | Média |### Baixa Prioridade (v3.1+)| # | Melhoria | Impacto | Complexidade ||---|----------|---------|--------------|| 7 | **Black-Litterman** | Views do investidor | Alta || 8 | **Aprimoramentos de UX** | Export CSV/HTML, tooltips | Baixa |### Dependências```[1] USD→BRL → nenhuma[2] yfinance → nenhuma[3] rapidfuzz → [2] se usar para tickers[4] Restrições → nenhuma[5] Backtesting → [1], [4][6] Ledoit-Wolf → nenhuma[7] Black-Litterman → [4], [6][8] UX → todas anteriores```---## ✅ Checklist de Validação (v2.5)### Bug de mapeamento corrigido:- [ ] Se aba foi excluída por MIN_OBS, status = "MAPEADO_EXCLUIDO_MIN_OBS"- [ ] Peso NÃO é remapeado para outro ativo- [ ] DataFrame de mapeamento mostra todos os status### Tabelas estruturadas:- [ ] Tabela de métricas (GMV, Max Sharpe, Atual) exibida- [ ] Tabela de pesos por ativo (comparativo) exibida- [ ] Top N ordenado por peso### Robustez:- [ ] Status de sucesso das otimizações reportado- [ ] Fallback para equal-weight se otimização falhar- [ ] MIN_OVERLAP aplicado na covariância